<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#让函数返回结果的技巧" data-toc-modified-id="让函数返回结果的技巧-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>让函数返回结果的技巧</a></span><ul class="toc-item"><li><span><a href="#单个函数不要返回多类型" data-toc-modified-id="单个函数不要返回多类型-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>单个函数不要返回多类型</a></span></li><li><span><a href="#使用partial来构造新的函数" data-toc-modified-id="使用partial来构造新的函数-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>使用partial来构造新的函数</a></span></li><li><span><a href="#让函数抛出异常而不是将结果和错误信息一起抛出" data-toc-modified-id="让函数抛出异常而不是将结果和错误信息一起抛出-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>让函数抛出异常而不是将结果和错误信息一起抛出</a></span></li><li><span><a href="#合理使用空对象模式来代替空值返回与抛出异常" data-toc-modified-id="合理使用空对象模式来代替空值返回与抛出异常-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>合理使用空对象模式来代替空值返回与抛出异常</a></span></li></ul></li><li><span><a href="#异常处理的好习惯" data-toc-modified-id="异常处理的好习惯-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>异常处理的好习惯</a></span><ul class="toc-item"><li><span><a href="#只做最精准的异常处理" data-toc-modified-id="只做最精准的异常处理-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>只做最精准的异常处理</a></span></li><li><span><a href="#别让异常破坏抽象一致性" data-toc-modified-id="别让异常破坏抽象一致性-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>别让异常破坏抽象一致性</a></span></li><li><span><a href="#异常处理不应该喧宾夺主" data-toc-modified-id="异常处理不应该喧宾夺主-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>异常处理不应该喧宾夺主</a></span></li></ul></li><li><span><a href="#编写地道的循环" data-toc-modified-id="编写地道的循环-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>编写地道的循环</a></span><ul class="toc-item"><li><span><a href="#使用函数修饰被迭代对象来优化循环" data-toc-modified-id="使用函数修饰被迭代对象来优化循环-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>使用函数修饰被迭代对象来优化循环</a></span></li><li><span><a href="#按职责拆解循环体内复杂代码块" data-toc-modified-id="按职责拆解循环体内复杂代码块-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>按职责拆解循环体内复杂代码块</a></span></li></ul></li></ul></div>

##### 让函数返回结果的技巧

###### 单个函数不要返回多类型

###### 使用partial来构造新的函数

In [16]:
def multiply(x, y):
    return x * y

from functools import partial
#double = partial(multiply, 2)
double = partial(multiply, x = 2)

double(y = 3)

6

###### 让函数抛出异常而不是将结果和错误信息一起抛出

###### 合理使用空对象模式来代替空值返回与抛出异常

In [19]:
import decimal


class CreateAccountError(Exception):
    pass


class NullAccount(object):
    username = ''
    balance = 0

    @classmethod
    def from_string(cls, s):
        raise NotImplementedError


class Account(object):

    def __init__(self, username, balance):
        self.username = username
        self.balance = balance

    @classmethod
    def from_string(cls, s):
        """从字符串初始化一个账号
        """
        try:
            username, balance = s.split()
            balance = decimal.Decimal(float(balance))
        except ValueError:
            raise CreateAccountError('input must follow pattern\
                                     "{ACCOUNT_NAME} {BALANCE}"')
        if balance < 0:
            #raise CreateAccountError('balance can not be negative')
            return NullAccount()
        return cls(username, balance)

In [20]:
#上面的代码中在判断balance为负数时没有抛出异常而是返回空对象.
#这样下面的函数得以简化
def calculate_total_balance(accounts_data):
    """计算所有账号的总的余额
    """
    return sum(Account.from_string(s).balance for s in accounts_data)

#否则在 calculate_total_balance里面你需要try/except:
def caculate_total_balance_(accounts_data):
    """计算所有账号的总余额
    """
    result = 0
    for account_string in accounts_data:
        try:
            user = Account.from_string(account_string)
        except CreateAccountError:
            pass
        else:
            result += user.balance
    return result

##### 异常处理的好习惯

###### 只做最精准的异常处理

* 永远只捕获那些可能会抛出异常的语句块
* 尽量只捕获精确的异常类型，而不是模糊的 Exception

In [21]:
from requests.exceptions import RequestException

def save_website_title(url, filename):
    try:
        resp = requests.get(url)
    except RequestException as e:
        print(f'save failed: unable to get page content: {e}')
        return False
    # 这段正则操作本身就是不应该抛出异常的，所以我们没必要使用 try 语句块
    # 假如 group 被误打成了 grop 也没关系，程序马上就会通过 AttributeError 来
    # 告诉我们。
    obj = re.search(r'<title>(.*)</title>', resp.text)
    if not obj:
        print('save failed: title tag not found in page content')
        return False
    title = obj.group(1)
    
    try:
        with open(filename, 'w') as fp:
            fp.write(title)
    except IOError as e:
        print(f'save failed: unable to write to file {filename}: {e}')
        return False
    else:
        return True
        

###### 别让异常破坏抽象一致性

* 让模块只抛出与当前抽象层级一致的异常
* 在贴近高层抽象的地方, 将低级层面的异常包装成高级异常

###### 异常处理不应该喧宾夺主

* 异常处理的逻辑太多可能扰乱代码的核心逻辑

In [22]:
#过多的异常处理
def upload_avatar(request):
    """用户上传新头像"""
    try:
        avatar_file = request.FILES['avatar']
    except KeyError:
        raise error_codes.AVATAR_FILE_NOT_PROVIDED

    try:
        resized_avatar_file = resize_avatar(avatar_file)
    except FileTooLargeError as e:
        raise error_codes.AVATAR_FILE_TOO_LARGE
    except ResizeAvatarError as e:
        raise error_codes.AVATAR_FILE_INVALID

    try:
        request.user.avatar = resized_avatar_file
        request.user.save()
    except Exception:
        raise error_codes.INTERNAL_SERVER_ERROR
    return HttpResponse({})

In [23]:
#解决方案: 上下文管理器简化 try/except

#这里我们没有用 contextlib模块
class raise_api_error(object):
    """捕捉特定的异常并且用error_code类来替代它.
    
    : raises: AttributeError if code_name is not valid
    """
    def __init__(self, captures, code_name):
        self.captures = captures
        self.code = getattr(error_codes, code_name)
        
    def __enter__(self):
        #该方法在进入上下文管理器时被调用
        return self
    def __exit__(self, exp_type, value, traceback):
        #该方法在退出上下文时调用
        #exp_type, value, traceback分别是该上下文抛出的
        #异常类型, 异常值和错误栈
        if exp_type is None:
            return False
        
        if exp_type == self.captures:
            raise self.code from value
        return False

In [ ]:
#在上面的代码里，我们定义了一个名为 raise_api_error 的上下文管理器，
#它在进入上下文时什么也不做。但是在退出上下文时，
#会判断当前上下文中是否抛出了类型为 self.captures 的异常，
#如果有，就用 APIErrorCode 异常类替代它。

def upload_avatar(request):
    """用户上传新头像"""
    with raise_api_error(KeyError, 'AVATAR_FILE_NOT_PROVIDED'):
        avatar_file = request.FILES['avatar']

    with raise_api_error(ResizeAvatarError, 'AVATAR_FILE_INVALID'),\
            raise_api_error(FileTooLargeError, 'AVATAR_FILE_TOO_LARGE'):
        resized_avatar_file = resize_avatar(avatar_file)

    with raise_api_error(Exception, 'INTERNAL_SERVER_ERROR'):
        request.user.avatar = resized_avatar_file
        request.user.save()
    return HttpResponse({})

##### 编写地道的循环

###### 使用函数修饰被迭代对象来优化循环

In [24]:
from itertools import product
#利用product来避免循环嵌套
def find_twelve(num_list1, num_list2, num_list3):
    for num1, num2, num3 in product(num_list1, num_list2, num_list3):
        if num1 + num2 + num3 == 12:
            return (num1, num2, num3)

In [ ]:
from itertools import islice
#利用islice(seq, start, end, step)完成切片操作
def parse_titles(filename):
    with open(filename, 'r') as fp:
        for line in islice(fp, 0, None, 2):
            yield line.strip()

In [25]:
from itertools import takewhile
#takewhile(predicate, iterable) 会在迭代 iterable 的过程中
#不断使用当前对象作为参数调用 predicate 函数并测试返回结果，
#如果函数返回值为真，则生成当前对象，循环继续。否则立即中断当前循环！！！
#来代替break语句

# for usr in takewhile(is_qualified, users):
#     pass

In [26]:
def even_only(numbers):
    return (num for num in numbers if num % 2 == 0)
#使用生成器来简化循环
def sum_even_only(numbers):
    result = 0
    for num in even_only(numbers):
        result += num
    return result

###### 按职责拆解循环体内复杂代码块

In [27]:
import time
import datetime

#外循环: 挑选日期与准备时间戳
#内循环: 发送奖励积分
#两个逻辑耦合在一起了, 不利于后期代码复用
def award_active_users_in_last_30days():
    """获取所有在过去 30 天周末晚上 8 点到 10 点登录过的用户，为其发送奖励积分
    """
    days = 30
    for days_delta in range(days):
        dt = datetime.date.today() - datetime.timedelta(days=days_delta)
        # 5: Saturday, 6: Sunday
        if dt.weekday() not in (5, 6):
            continue

        time_start = datetime.datetime(dt.year, dt.month, dt.day, 20, 0)
        time_end = datetime.datetime(dt.year, dt.month, dt.day, 23, 0)

        # 转换为 unix 时间戳，之后的 ORM 查询需要
        ts_start = time.mktime(time_start.timetuple())
        ts_end = time.mktime(time_end.timetuple())

        # 查询用户并挨个发送 1000 奖励积分
        for record in LoginRecord.filter_by_range(ts_start, ts_end):
            # 这里可以添加复杂逻辑
            send_awarding_points(record.user_id, 1000)

In [28]:
#使用生成器函数解耦循环
#外循环剥离出来
def gen_weekend_ts_ranges(days_ago, hour_start, hour_end):
    """生成过去一段时间内周六日特定时间段范围，并以 UNIX 时间戳返回
    """
    for days_delta in range(days_ago):
        dt = datetime.date.today() - datetime.timedelta(days=days_delta)
        if dt.weekday() not in (5, 6):
            continue
        
        t_start = datetime.datetime(dt.year, dt.month,
                                    dt.day, hour_start, 0)
        t_end = datetime.datetime(dt.year, dt.month,
                                  dt.day, hour_end, 0)
        
        #转换为UNIX时间戳, 为之后的ORM做准备
        t_start = time.mktime(t_start.timetuple())
        t_end = time.mktime(t_end.timetuple())
        yield t_start, t_end

In [29]:
#有了上面的生成器之后, 需求“发送奖励积分”, 都可以再循环体内复用他来完成任务
def award_active_users_in_last_30days():
    for ts_start, ts_end in gen_weekend_ts_ranges(30, hour_start=20,
                                                  hour_end=23):
        for record in LoginRecord.filter_by_range(ts_start, ts_end):
            send_award_points(record.user_id, 1000)